In [ ]:
# rules of the game
"""
1) Equipment :
3 red dice for attack
3 blue dice for defense
44 risk cards 'including 2 jokers: LATER'and 42 territories.
14 mission cards: LATER
No limit of troop

2) preliminary set up of game
Choose of troops depending of number of players => 3 players = 35 troops.....
distribute all the territories randomly to the players and add exactly one troop of the player  on the territory.
put the remaining troops in the stock of each Player


Game for 2 players rules with neutral army: LATER
40 troops per Player
28 troops for the neutral army
14 territories for the neutral army
14 territories for each Player.
2 troops on the territories of the neutral army
1 troop on each territory of each player

3)Who goes first? ==> give a player number 

4)The game itself
    a)Additionnal set up of the game:
Player 1 add 1 troop on one of its territories 
Player 2 add 1 troop on one of its territories ....and so on until all the troops are on the boardgame.
    b) Let's begin the actual game
Phase 1: reinforcement
    Get on 1 troop for every 3 territories you own
    Get bonus troops if you own the whole continent. Depends on the continent.
    Get extra troops depending of the combinations of the symbols on 3 risk cards you play : 3 infantry: 4 troops, 3 cavalry: 6 troops, 3 artillery: 8 troops, one of each :10 troops. start with only one or 2 combinations.
    Get extra troops depending of the territories on the risk cards you play: 2 extra troops if you are the owner of the territory at the moment you play
   f 
Phase 2: Attack
    Announce which territory attacks which territory. Be sure you take a atttacking territories from the territories you are occupying, and that those 2 territories are
    actually neighbours.
    Check if the destination territory does not belong to the player and if the number of troops is higher than 1 'no empty territory rule'
    decide our many troops you want to commit for the attack
    make sure that the number of troops attacking is <4 LATER
    roll the 3 dices on both sides and compared the max of each side . The max wins and the min loses one troop . 
    Automatically roll the dices until there a strict difference between the dice.
    maybe LATER : If the player attacking wins one or more territories : he wins 1 'territory cards '. Max 5 cards in the player hands==> he has to play them when its his turn and amount exceed 3.
    the player can keep attacking as long as he wants or until he cannot play 'loss or only territories with one troop'
    Display the board after every action

Phase 3: Manoeuvre
    Can move only one group of troops from one territory to another . The territories have to belong to the player and no empty territory rule.
    'move function to be create to handle phase 3 and phase 2 too'
    
5) Engame
Either Player conquers at least 30 territories.
    
"""

In [7]:
class Country:
    
    def __init__(self,country_name):
        self.name=country_name
        self.color='White'
        self.nbtroops=0
     
    #udpate the color of the coutry with the color of the player which occupied it.
    def update_color_of_country(self,Player_obj):
        self.color=Player_ojb.color
        return self.color
    
    #udpate the number of troops which are in the country.
    def update_nb_troops(self,new_troop_nb):
        self.nbtroops=new_troop_nb
        return self.nbtroops
        

In [6]:
class Continent:
    
    def __init__(self,lst_countries_of_continent,bonus_of_continent):
        self.bonus=bonus_of_continent
        self.countries=lst_countries_of_continent
    
    # Function which check whether Player's color is on all the countries of the continent
    def is_eligible_to_bonus(self):
        pass
    
    # Function which provides the value of the bonus if the continent is fully occupied by Player.
    def get_value_of_bonus(self):
        return self.bonus
    
    
    
    

In [ ]:
#class MAP to create the boardgame with functions to check whether the boardgame created manually or automatically makes sense
class Map:
    def __init__(self,limits,lst_continents):
        self.frontiers=limits
        self.allcontinents=lst_continents
        self.allcountries=[Pays, for Pays in ]
    
    def get_all_continents(self):
        self.al
        pass
   
    def get_all_countries(self):
        self.allcountries=[Pays for Pays in]
        pass
    
    def Check_limits(self,limits,self.allcountries):
        pass
        
    

In [9]:
France=Country('France')
Italy=Country('Italy')
Spain=Country('Spain')
Morocco=Country('Morocco')
Senegal=Country('Senegal')

In [10]:
frontiers=[('France','Italy'), ('France','Spain'), ('Morocco','Spain')]

In [11]:
Europe=Continent([France,Spain,Italy],5) 
Africa=Continent([Morocco,Senegal],3)

In [209]:
  #dictionnary: territories with their ID
        
territory_id={
    1: "Alaska", 2: "Alberta", 3: "Central America", 4: "Eastern United States", 5: "Greenland",
    6: "Northwest Territory", 7: "Ontario", 8: "Quebec", 9: "Western United States",10: "Argentina", 
    11: "Brazil", 12: "Peru", 13: "Venezuela", 14: "Great Britain", 15: "Iceland", 
    16: "Northern Europe", 17: "Scandinavia", 18: "Southern Europe", 19: "Ukraine", 20: "Western Europe",
    21: "Congo", 22: "East Africa", 23: "Egypt", 24: "Madagascar", 25: "North Africa",
    26: "South Africa", 27: "Afghanistan", 28: "China", 29: "India", 30: "Irkutsk",
    31: "Japan", 32: "Kamchatka", 33: "Middle East", 34: "Mongolia", 35: "Siam ",
    36: "Siberia", 37: "Ural", 38: "Yakutsk", 39: "Eastern Australia", 40: "Indonesia",
    41: "New Guinea",42: "Western Australia"
} 
     #list of territories 
        #dictionnary: territories with their ID, the ID of the player occupying the territory, the Nb of troops on the territory
territory_occupation={
    1: [1, 2],
    2: ["", 0],
    3: ["", 0],
    4: ["", 0],
    5: ["", 0],
    6: ["", 0],
    7: ["", 0],
    8: ["", 0],
    9: ["", 0],
    10: ["", 0],
    11: ["", 0],
    12: ["", 0],
    13: ["", 0],
    14: ["", 0],
    15: ["", 0],
    16: ["", 0],
    17: ["", 0],
    18: ["", 0],
    19: ["", 0],
    20: ["", 0],
    21: ["", 0],
    22: ["", 0],
    23: ["", 0],
    24: ["", 0],
    25: ["", 0],
    26: ["", 0],
    27: ["", 0],
    28: ["", 0],
    29: ["", 0],
    30: ["", 0],
    31: ["", 0],
    32: ["", 0],
    33: ["", 0],
    34: ["", 0],
    35: ["", 0],
    36: ["", 0],
    37: ["", 0],
    38: ["", 0],
    39: ["", 0],
    40: ["", 0],
    41: ["", 0],
    42: ["", 0],
}
    
     #Dictionnary: list of continents with the territories in each continent
continent_id={
    "North America":  [1, 2, 3, 4, 5, 6, 7, 8, 9],
    "South America": [10, 11, 12, 13],
    "Europe": [14, 15, 16, 17, 18, 19, 20],
    "Africa": [21, 22, 23, 24, 25, 26],
    "Asia": [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38],
    "Australia": [39, 40, 41, 42]    
}
    
    #Dictionnary: list of the territories with the borders (neighbour territories)
neighour_id={
    1: [2, 6, 32],
    2: [1, 6, 7, 9],
    9: [2, 3, 4, 7],
    10: [11, 12],
    11: [10, 12, 13, 25],
    12: [10, 11, 13],
    13: [3, 11, 12],
    14: [15, 16, 17, 20],
    15: [5, 16, 17],
    16: [14, 17, 18, 19, 20],
    17: [14, 15, 16, 19],
    18: [16, 19, 20, 23, 25, 33],
    19: [16, 17, 27, 33, 37],
    20: [14, 16, 18, 25],
    21: [22, 25, 26],
    22: [21, 23, 24, 25, 26, 33],
    23: [18, 22, 25, 33],
    24: [22, 26],
    25: [11, 18, 20, 21, 22, 23],
    26: [21, 22, 24],
    27: [19, 28, 29, 33, 37],
    28: [27, 29, 34, 35, 36, 37],
    29: [27, 28, 33, 35],
    30: [32, 34, 36, 38],
    31: [32, 34],
    32: [1, 30, 31, 34, 38],
    33: [18, 19, 22, 23, 27, 29],
    34: [28, 30, 31, 32, 36],
    35: [28, 29, 40],
    36: [28, 30, 34, 37, 38],
    37: [19, 27, 28, 36],
    38: [30, 32, 36],  
    39: [41, 42],
    40: [35, 41, 42],
    41: [39, 40, 42],
    42: [39, 40, 41]
}
  
continent_bonus={
    "North America":  3,
    "South America": 1,
    "Europe": 2,
    "Africa": 2,
    "Asia": 5,
    "Australia": 1    
}

In [211]:
class Player:

    def __init__(self,id_player):
        self.name='' #name_player
        self.id=id_player  # position of the player in the game
        self.occupied_territories=dict() # dictionnary with the territories owned and the number of troops on each territory
        self.riskcards=[]#Later maybe
        self.missions=[] #Later maybe
        self.stock=0 # counter for the reinforcement phase
        
        
     #provide the list of territories occupied by the player 
    def get_occupied_territories(self):
        self.occupied_territories={x:territory_occupation[x][1] for x in territory_occupation.keys() if territory_occupation[x][0]==self.id}
        return self.occupied_territories
   
    def continent_check(self):
        progress={i: set(j).issubset(self.occupied_territories.keys()) for i,j in continent_id.items()}
        return progress
  
    def count_nb_troop_to_place(self):
        #owned territory bonus
        bonus_nb_territories=int((len(list(self.occupied_territories.keys())))/3)
    
        #continent bonus
        owned_continent=self.continent_check().values()
        if sum(owned_continent)>0:
            for i in continent_check.keys():
                if continent_check[i]==True:
                    bonus_continent+=continent_bonus[i]
        else:
            bonus_continent=0
        reinforcement_troop=bonus_nb_territories+ bonus_continent
        return reinforcement_troop
     
    def reinforce_troop(self):
        self.count_nb_troop_to_place()
        while self.count_nb_troop_to_place()!=0: 
            territory=random.choice(list(player.occupied_territories.keys()))
            player.occupied_territories[territory]+=player.occupied_territories[territory]
            territory_occupation[territory][1]+=1
            self.count_nb_troop_to_place()-=1
            self.stock-=self.count_nb_troop_to_place()

   
        

SyntaxError: can't assign to function call (<ipython-input-211-13499789dddd>, line 42)

In [212]:
Joueur1=Player(1)
Joueur2=Player(2)

In [213]:

#Who goes first? ==> give a player number 
lst_player=['Alice','Bob']
random.shuffle(lst_player)
Joueur1.name=lst_player[0]
Joueur2.name=lst_player[1]

In [214]:


#for 2 players / to be adapted for more players
Nb_player=2
#choose of Number of troops in each game depending on the number of players
def game_setup(NbPlayer):
    army_size={2:40,3:35,4:30,5:35,6:20}
    return army_size[NbPlayer]

def assign_territories(Nb_player):
    #shuffle the territories
    import random
    deck_of_territories=random.sample(territory_id.keys(),len(territory_id.keys()))

    #distribute all the territories randomly to the players and add exactly one troop of the player  on the territory.

    for i,territory in enumerate(deck_of_territories):
        player=Joueur1
        if i%2==0:
            player=Joueur2   
        player.occupied_territories[territory]=1
        #Update the territory_occupation
        territory_occupation[i]=[player.id,1]
    #Update stock of remaining Troops for each player
    Joueur1.stock=game_setup(Nb_player)-len(list(Joueur1.occupied_territories.keys()))
    Joueur2.stock=game_setup(Nb_player)-len(list(Joueur2.occupied_territories.keys())) 
    #dispatch remaining troops randomly on the occupied territories for each player
    while Joueur1.stock!=0 or Joueur2.stock!=0: 
        for player in [Joueur1, Joueur2]:
            if player.stock != 0:
                territory=random.choice(list(player.occupied_territories.keys()))
                player.occupied_territories[territory]+=player.occupied_territories[territory]
                territory_occupation[territory][1]+=1
                player.stock -=1
    return 


assign_territories(Nb_player)
print(Joueur2.stock)
print(Joueur1.stock)
print(Joueur2.occupied_territories.keys())
print(Joueur1.occupied_territories.keys())
print(territory_occupation)


0
0
dict_keys([42, 13, 21, 8, 6, 19, 15, 10, 22, 37, 20, 17, 38, 31, 26, 28, 34, 40, 25, 36, 27])
dict_keys([12, 18, 3, 11, 30, 32, 29, 41, 4, 16, 23, 33, 5, 14, 1, 9, 2, 35, 24, 39, 7])
{1: [1, 5], 2: [2, 2], 3: [1, 2], 4: [2, 1], 5: [1, 1], 6: [2, 2], 7: [1, 3], 8: [2, 2], 9: [1, 2], 10: [2, 1], 11: [1, 5], 12: [2, 2], 13: [1, 2], 14: [2, 2], 15: [1, 1], 16: [2, 1], 17: [1, 2], 18: [2, 1], 19: [1, 3], 20: [2, 3], 21: [1, 1], 22: [2, 1], 23: [1, 1], 24: [2, 1], 25: [1, 1], 26: [2, 3], 27: [1, 4], 28: [2, 4], 29: [1, 1], 30: [2, 1], 31: [1, 1], 32: [2, 1], 33: [1, 3], 34: [2, 2], 35: [1, 2], 36: [2, 2], 37: [1, 2], 38: [2, 1], 39: [1, 1], 40: [2, 1], 41: [1, 2], 42: ['', 0], 0: [2, 1]}


In [192]:



"""
4)The game itself
    a)Additionnal set up of the game:
Player 1 add 1 troop on one of its territories 
Player 2 add 1 troop on one of its territories ....and so on until all the troops are on the boardgame.

while Joueur1.stock!=0 or Joueur2.stock!=0: 
    if Joueur1.stock==0 and Joueur2.stock!=0 :
        print('Player 1, no more troop to place, wait for the beginning of the game')
        print('Player number 2, please place a troop on one of your territories.')
        territory2=int(input(f'Your current territories are {Joueur2.occupied_territories}.What is your choice ?:'))
        if territory2 not in Joueur2.occupied_territories.keys():         
            print(f'this territory is not part of your territories, please choose a territory among {Joueur2.occupied_territories.keys()}')
            territory2=int(input(f'Your current territories are {Joueur2.occupied_territories}.What is your choice ?:'))     
        Joueur2.occupied_territories[territory2]+=Joueur2.occupied_territories[territory2]
        territory_occupation[territory2][1]+=1
        
    if Joueur1.stock!=0 and Joueur2.stock==0 :
        print('Player2, no more troop to place, wait for the beginning of the game')
        print('Player number 1, please place a troop on one of your territories.')
        territory1=int(input(f'Your current territories are {Joueur1.occupied_territories}.What is your choice ?:'))
        if territory1 not in Joueur21.occupied_territories.keys():         
            print(f'this territory is not part of your territories, please choose a territory among {Joueur1.occupied_territories.keys()}')
            territory1=int(input(f'Your current territories are {Joueur1.occupied_territories}.What is your choice ?:'))
        Joueur1.occupied_territories[territory1]+=Joueur1.occupied_territories[territory1]
        territory_occupation[territory1][1]+=1
        
    if Joueur1.stock!=0 and Joueur2.stock!=0:
        print('Player number 1, please place a troop on one of your territories.')
        territory1=int(input(f'Your current territories are {Joueur1.occupied_territories}.What is your choice ?:'))
        if territory1 not in Joueur1.occupied_territories.keys():         
            print(f'this territory is not part of your territories, please choose a territory among {Joueur1.occupied_territories.keys()}')
            territory1=int(input(f'Your current territories are {Joueur1.occupied_territories}.What is your choice ?:'))
        Joueur1.occupied_territories[territory1]+=Joueur1.occupied_territories[territory1]
        territory_occupation[territory1][1]+=1
        
        print('Player number 2, please place a troop on one of your territories.')
        territory2=int(input(f'Your current territories are {Joueur2.occupied_territories}.What is your choice ?:'))
        if territory2 not in Joueur2.occupied_territories.keys():         
            print(f'this territory is not part of your territories, please choose a territory among {Joueur2.occupied_territories.keys()}')
            territory2=int(input(f'Your current territories are {Joueur2.occupied_territories}.What is your choice ?:'))
        Joueur2.occupied_territories[territory2]+=Joueur2.occupied_territories[territory2]
        territory_occupation[territory2][1]+=1
    else:
        print('No more troop to place on the board. Let\'s start to play!!')

   
#random choice of the country

"""


"\n4)The game itself\n    a)Additionnal set up of the game:\nPlayer 1 add 1 troop on one of its territories \nPlayer 2 add 1 troop on one of its territories ....and so on until all the troops are on the boardgame.\n\nwhile Joueur1.stock!=0 or Joueur2.stock!=0: \n    if Joueur1.stock==0 and Joueur2.stock!=0 :\n        print('Player 1, no more troop to place, wait for the beginning of the game')\n        print('Player number 2, please place a troop on one of your territories.')\n        territory2=int(input(f'Your current territories are {Joueur2.occupied_territories}.What is your choice ?:'))\n        if territory2 not in Joueur2.occupied_territories.keys():         \n            print(f'this territory is not part of your territories, please choose a territory among {Joueur2.occupied_territories.keys()}')\n            territory2=int(input(f'Your current territories are {Joueur2.occupied_territories}.What is your choice ?:'))     \n        Joueur2.occupied_territories[territory2]+=Joueur

In [183]:
"""while Joueur1.stock!=0 or Joueur2.stock!=0: 
    for player in [Joueur1, Joueur2]:
        if player.stock != 0:
            print(f'Player number {player.id}, please place a troop on one of your territories.')
            territory=int(input(f'Your current territories are {player.occupied_territories}.What is your choice ?:'))
            while territory not in player.occupied_territories.keys():         
                print(f'this territory is not part of your territories, please choose a territory among {player.occupied_territories.keys()}')
                territory=int(input(f'Your current territories are {player.occupied_territories}.What is your choice ?:'))
            player.occupied_territories[territory]+=player.occupied_territories[territory]
            territory_occupation[territory][1]+=1

"""

"while Joueur1.stock!=0 or Joueur2.stock!=0: \n    for player in [Joueur1, Joueur2]:\n        if player.stock != 0:\n            print(f'Player number {player.id}, please place a troop on one of your territories.')\n            territory=int(input(f'Your current territories are {player.occupied_territories}.What is your choice ?:'))\n            while territory not in player.occupied_territories.keys():         \n                print(f'this territory is not part of your territories, please choose a territory among {player.occupied_territories.keys()}')\n                territory=int(input(f'Your current territories are {player.occupied_territories}.What is your choice ?:'))\n            player.occupied_territories[territory]+=player.occupied_territories[territory]\n            territory_occupation[territory][1]+=1\n\n"

KeyboardInterrupt: 

In [ ]:
"""
    b) Let's begin the actual game
Phase 1: reinforcement
    Get on 1 troop for every 3 territories you own
    Get bonus troops if you own the whole continent. Depends on the continent.
    Get extra troops depending of the combinations of the symbols on 3 risk cards you play : 3 infantry: 4 troops, 3 cavalry: 6 troops, 3 artillery: 8 troops, one of each :10 troops. start with only one or 2 combinations.
    3 infantry: 4 troops, 3 cavalry: 6 troops, 3 artillery: 8 troops, one of each :10 troops
    Get extra troops depending of the territories on the risk cards you play: 2 extra troops if you are the owner of the territory at the moment you play
   f """


In [ ]:
"""
 
risk_cards={
    1, "infantry", 2: "artillery", 3: "cavalry", 4: "infantry", 5: "artillery",
    6: "cavalry", 7: "infantry", 8: "artillery", 9: "cavalry",10: "infantry", 
    11: "artillery", 12: "cavalry", 13: "cavalry", 14: "artillery", 15: "cavalry", 
    16: "cavalry", 17: "artillery", 18: "infantry", 19: "artillery", 20: "cavalry",
    21: "infantry", 22: "artillery", 23: "cavalry", 24: "infantry", 25: "artillery",
    26: "cavalry", 27: "infantry", 28: "artillery", 29: "cavalry", 30: "infantry",
    31: "artillery", 32: "cavalry", 33: "infantry", 34: "artillery", 35: "cavalry ",
    36: "infantry", 37: "artillery", 38: "cavalry", 39: "infantry", 40: "artillery",
    41: "cavalry",42: "infantry"
}

Bonus_risk_card_combination={"infantry":4, "cavalry": 6, "artillery": 8, "OneofEach":10}"""